# Topic modeling

In [ ]:
import logging
import math
import os
import sys
from imp import reload
from time import time

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import cauchy, norm

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import normalize, Normalizer
import sklearn.mixture as gmm

import gensim
import wordcloud


module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

from event_detection import data_fetchers, event_detector

reload(logging)
logging.basicConfig(stream=sys.stdout, format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


%matplotlib inline

# 1. Fetch data
(only the Dec-Jan subset for now)

In [ ]:
total_time = time()

t = time()
documents, relative_days = data_fetchers.fetch_czech_corpus_dec_jan()

stream_length = max(relative_days) + 1  # Zero-based, hence the + 1.
print('Read input in %fs.' % (time() - t))
print('Stream length: %d' % stream_length)

t = time()
vectorizer = CountVectorizer(min_df=30, max_df=100000, binary=True, stop_words=event_detector.CZECH_STOPWORDS)
bow_matrix = vectorizer.fit_transform(documents).tocsr()
id2word = {v: k for k, v in vectorizer.vocabulary_.items()}
print('Created bag of words in %fs.' % (time() - t))
print('BOW:', bow_matrix.shape)

# 2. Detect events

In [ ]:
trajectories = event_detector.construct_feature_trajectories(bow_matrix, relative_days)
dps, dp = event_detector.spectral_analysis(trajectories)

# Aperiodic events
aperiodic_indices = np.where((dps > event_detector.DPS_BOUNDARY) & (dp > math.ceil(stream_length / 2)))[0]
aperiodic_bow = bow_matrix[:, aperiodic_indices]
aperiodic_features = trajectories[aperiodic_indices, :]
aperiodic_dps = dps[aperiodic_indices]
aperiodic_dp = dp[aperiodic_indices]
aperiodic_events = event_detector.unsupervised_greedy_event_detection(aperiodic_indices, aperiodic_bow, aperiodic_features,
                                                       aperiodic_dps, aperiodic_dp)

# Periodic events
periodic_indices = np.where((dps > event_detector.DPS_BOUNDARY) & (dp <= math.ceil(stream_length / 2)))[0]
periodic_bow = bow_matrix[:, periodic_indices]
periodic_features = trajectories[periodic_indices, :]
periodic_dps = dps[periodic_indices]
periodic_dp = dp[periodic_indices]
periodic_events = event_detector.unsupervised_greedy_event_detection(periodic_indices, periodic_bow, periodic_features,
                                                      periodic_dps, periodic_dp)

Generators to lists:

In [ ]:
%%time
aperiodic_events = list(aperiodic_events)
periodic_events = list(periodic_events)

# 3. Perform TFIDF rescaling
Actually, just IDF rescaling, since the documents are preprocessed to contain each word just once. Nevertheless, this means a *huge* improvement. All methods generally perform better, some even faster, and there are even some new document types uncovered.

In [ ]:
%time bow_matrix = TfidfTransformer().fit_transform(bow_matrix)

In [ ]:
NUM_TOPICS = 100

# 4. Gensim models

In [ ]:
%%time
corpus = gensim.matutils.Sparse2Corpus(bow_matrix, documents_columns=False)
dictionary = gensim.corpora.Dictionary.from_corpus(corpus, id2word=id2word)

## LSI model
Fitting takes about 1 minute for the Dec-Jan dataset with 10 topics and about 4 minutes for 100 topics.

In [ ]:
LSI_PATH = ('./dec_jan_%d_topics_tfidf.lsi' % NUM_TOPICS)

if os.path.exists(LSI_PATH):
    lsi = gensim.models.LsiModel.load(LSI_PATH)
    print('Loaded %d LSI topics from file' % NUM_TOPICS)
else:
    %time lsi = gensim.models.LsiModel(corpus, id2word=dictionary, num_topics=NUM_TOPICS, onepass=False, power_iters=5)
    lsi.save(LSI_PATH)
    print('Generated LSI model for %d topics and saved to file' % NUM_TOPICS)

Project the documents into the latent space (taken from Gensim GitHub FAQ) and normalize to unit l2 norm, which causes KMeans to behave as spherical KMeans:

In [ ]:
%%time
lsi_gensim = gensim.matutils.corpus2dense(lsi[corpus], len(lsi.projection.s)).T / lsi.projection.s
normalize(lsi_gensim, norm='l2', copy=False)

In [ ]:
lsi.print_topics(10)

## LDA model
Fitting takes about 15 minutes for the Dec-Jan dataset with 10 topics.

In [ ]:
LDA_PATH = ('./dec_jan_%d_topics.lda' % NUM_TOPICS)

if os.path.exists(LDA_PATH):
    lda = gensim.models.LdaModel.load(LDA_PATH)
    print('Loaded %d LDA topics from file' % NUM_TOPICS)
else:
    %time lda = gensim.models.LdaModel(corpus, id2word=dictionary, num_topics=NUM_TOPICS)
    lda.save(LDA_PATH)
    print('Generated LDA model for %d topics and saved to file' % NUM_TOPICS)

In [ ]:
lda.print_topics(10)

### Visualize LDA

In [ ]:
# import pyLDAvis
# import pyLDAvis.gensim

# pyLDAvis.enable_notebook()
# pyLDAvis.gensim.prepare(lda, corpus, dictionary)

### Create indexed corpus
Need to create an indexed corpus, otherwise corpus2csc is REALLY slow. The MM corpus itself took a long time to compute, and while the resulting clusters are a bit better, they are not good enough to justify the insane waiting time.

This is an error - we use sparse matrix to represent dense data. When fixed, this step may actually be faster.

In [ ]:
LDA_CORPUS_PATH = './lda_corpus.mm'

if os.path.exists(LDA_CORPUS_PATH):
    lda_corpus = gensim.corpora.MmCorpus('./lda_corpus.mm')
else:
    lda_corpus = lda[corpus]
    gensim.corpora.MmCorpus.serialize('./lda_corpus.mm', lda_corpus)

sparse_corpus = gensim.matutils.corpus2csc(lda_corpus)

# 5. Scikit-learn models
Sklearn's LDA model is even slower than the Gensim one.

## LSI model
Fitting takes only about 12 seconds (using randomized SVD) and 8 seconds (using ARPACK SVD) for the Dec-Jan dataset with 10 topics. Normalization to unit l2 norm causes KMeans to behave as spherical KMeans.

In [ ]:
svd = TruncatedSVD(n_components=NUM_TOPICS, algorithm='randomized')
normalizer = Normalizer(norm='l2', copy=False)
lsi_sklearn_model = make_pipeline(svd, normalizer)

%time lsi_sklearn = lsi_sklearn_model.fit_transform(bow_matrix)
print(lsi_sklearn.shape)

## NMF model
Takes about 4 minutes for the Dec-Jan dataset with 10 topics.

In [ ]:
nmf_model = NMF(n_components=NUM_TOPICS)
%time nmf = nmf_model.fit_transform(bow_matrix)
print(nmf.shape)

# 6. Document clustering
Times for full KMeans (without TFIDF) and 10 topics and clusters:
* Gensim LSI: about 40 seconds
* Gensim LDA: about 4 minutes
* Scikit-learn LSI: about 1 minute
* NMF: about 30 seconds

Times for MiniBatchKMeans are at most 2 seconds. The resulting clusters have only minor differences (this agrees with the documentation).

In [ ]:
NUM_CLUSTERS = 15
method = 'lsi_gensim'
# kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=1)
kmeans = MiniBatchKMeans(n_clusters=NUM_CLUSTERS, n_init=10, random_state=1)

if method == 'lsi_gensim':
    model = lsi_gensim
elif method == 'lda_gensim':
    model = sparse_corpus
elif method == 'lsi_sklearn':
    model = lsi_sklearn
elif method == 'nmf_sklearn':
    model = nmf
else:
    print('Invalid method')

%time kmeans.fit(model)

clusters = [[] for _ in range(NUM_CLUSTERS)]

for doc, label in np.ndenumerate(kmeans.labels_):
    clusters[label].append(doc[0])

## Visualize clusters

In [ ]:
cluster_words = [[] for _ in range(NUM_CLUSTERS)]

for i, cluster in enumerate(clusters):
    for doc in cluster:
        cluster_words[i].extend(documents[doc].split(r'\s'))

In [ ]:
for i, words in enumerate(cluster_words):
    wc = wordcloud.WordCloud(width=2000, height=1200).generate(' '.join(words))
    fig = plt.figure()
    fig.set_size_inches(20, 12)
    plt.imshow(wc)
    plt.axis('off')
    wc.to_file('./sklearn_lsi_15_topics_15_clusters/%02d.png' % i)

# 7. Get event trajectories and bursty periods

## Aperiodic

In [ ]:
def gaussian_curve(value, loc, scale):
    return norm.pdf(value, loc=loc, scale=scale)

aperiodic_parameters = []  # List of triples (event_id, event_mean, event_std).

for i, event in enumerate(aperiodic_events):
    # Create trajectory.
    e_trajectory, e_dominant_period = event_detector.create_event_trajectory(event, trajectories, dps, dp)
    
    # Calculate moving average & cutoff.
    ma = event_detector.moving_average(e_trajectory, event_detector.WINDOW)
    ma_mean = np.mean(ma)
    ma_std = np.std(ma)
    cutoff = ma_mean + ma_std
    
    # Fit Gaussian curve.
    n_days = len(e_trajectory)
    days = np.arange(n_days)
    
    peak_indices = np.where(e_trajectory > cutoff)
    peak_days = peak_indices[0]
    peaks = e_trajectory[peak_indices].reshape(-1)
    peaks /= np.sum(peaks)  # Normalize the DFIDF so it can be interpreted as probability.

    p0 = (peak_days[len(peak_days) // 2], len(peak_days) / 4)
    popt, pcov = curve_fit(gaussian_curve, peak_days, peaks, p0=p0, bounds=(0.0, n_days))
    
    # Extract parameters.
    mean, std = popt
    aperiodic_parameters.append((i, mean, std))
    
    # Plot some graphs.
    if i % 10 == 0:
        x = np.linspace(0.0, n_days, 1000)
        plt.title('Bursty period: (%d, %d)' % (math.floor(mean - std), math.ceil(mean + std)))
        plt.xlim(0.0, n_days)
        plt.hlines(cutoff / np.sum(e_trajectory), 0, n_days, colors='g', linestyles='dashed', linewidth=1.5)
        plt.axvline(math.floor(mean - std), color='r')
        plt.axvline(math.ceil(mean + std), color='r')
        plt.plot(days, e_trajectory / np.sum(e_trajectory))
        plt.plot(x, norm.pdf(x, mean, std))
        plt.show()

print('Found %d aperiodic events' % len(aperiodic_parameters))

## Periodic

In [ ]:
periodic_parameters = []  # List of pairs (event_id, [(event_loc, event_scale)]).

for i, event in enumerate(periodic_events):
    # Create trajectory.
    e_trajectory, e_dominant_period = event_detector.create_event_trajectory(event, trajectories, dps, dp)

    # Calculate moving average & cutoff.
    ma = event_detector.moving_average(e_trajectory, event_detector.WINDOW)
    ma_mean = np.mean(ma)
    ma_std = np.std(ma)
    cutoff = ma_mean + ma_std

    n_days = len(e_trajectory)
    days = np.arange(n_days)

    observations = np.hstack((days.reshape(-1, 1), e_trajectory.reshape(-1, 1)))
    observations = observations[observations[:, 1] > cutoff, :]
    normalized_trajectory = e_trajectory / np.sum(e_trajectory)

    # Fit mixture model.
    n_components = int(min(np.floor(n_days / e_dominant_period), len(observations)))
    g = gmm.GaussianMixture(n_components=n_components, covariance_type='diag')
    g.fit(observations)

    e_parameters = []
    
    # Extract parameters.
    for mean_, cov_ in zip(g.means_, g.covariances_):
        loc = mean_[0]
        hwhm = np.sqrt(2 * np.log(2)) * np.sqrt(cov_[0])
        
        e_parameters.append((loc, hwhm))

    periodic_parameters.append((i, e_parameters))
    
    # Plot some graphs.
    if i % 10 == 0:
        x = np.linspace(0.0, n_days, 1000)
        components = np.squeeze(np.array(
            [cauchy.pdf(x, mean[0], np.sqrt(2 * np.log(2)) * np.sqrt(cov[0])) for mean, cov in zip(g.means_, g.covariances_)]))
        
        pdf = g.weights_ @ components

        plt.title('DP: %d, n_components: %d' % (e_dominant_period, n_components))
        plt.xlim(0.0, n_days)
        plt.hlines(cutoff / np.sum(e_trajectory), 0, n_days, colors='g', linestyles='dashed', linewidth=1.5)
        plt.plot(days, e_trajectory / np.sum(e_trajectory))
        plt.plot(x, pdf)
        plt.show()

print('Found %d periodic events' % len(periodic_parameters))

# 8. Get event documents

## Aperiodic

In [ ]:
n_samples, n_features = bow_matrix.shape
dtd = event_detector.construct_doc_to_day_matrix(n_samples, relative_days)
aperiodic_documents = []  # List of pairs (event_id, array_of_event_document_ids).

for event, event_parameters in zip(aperiodic_events, aperiodic_parameters):
    event_id, event_mean, event_std = event_parameters

    burst_start = max(math.floor(event_mean - event_std), 0)  # If an event burst starts right at day 0, this would get negative.
    burst_end = min(math.ceil(event_mean + event_std), stream_length - 1)  # If an event burst ends at stream length, this would exceed the boundary.

    # Documents published on burst days.
    docs_dates, _ = dtd[:, burst_start:burst_end + 1].nonzero()  # There is exactly one '1' in every row.

    # Documents containing at least one of the event word features.
    docs_either_words = bow_matrix[:, event]

    # Documents containing all of the event word features.
    docs_words = np.where(docs_either_words.getnnz(axis=1) == len(event))[0]

    # Documents both published on burst days and containing all event word features.
    docs_both = np.intersect1d(docs_dates, docs_words, assume_unique=True)
    
    aperiodic_documents.append((event_id, docs_both))

## Periodic

In [ ]:
n_samples, n_features = bow_matrix.shape
dtd = event_detector.construct_doc_to_day_matrix(n_samples, relative_days)
periodic_documents = []  # List of pairs (event_id, array_of_event_document_ids).

for event, event_parameters in zip(periodic_events, periodic_parameters):
    event_id, locs_n_scales = event_parameters
    
    docs_dates = []

    for loc, scale in locs_n_scales:
        burst_start = max(math.floor(loc - scale), 0)
        burst_end = min(math.ceil(loc + scale), stream_length - 1)
        
        burst_dates, _ = dtd[:, burst_start:burst_end + 1].nonzero()
        docs_dates.extend(burst_dates.tolist())
    
    # Documents containing at least one of the event word features.
    docs_either_words = bow_matrix[:, event]

    # Documents containing all of the event word features.
    docs_words = np.where(docs_either_words.getnnz(axis=1) == len(event))[0]
    
    # Documents both published on burst days and containing all event word features.
    # Do not assume unique, as some bursty periods may overlap and fetch the same document twice.
    docs_both = np.intersect1d(docs_dates, docs_words, assume_unique=False)
    
    periodic_documents.append((event_id, docs_both))

# 9. Event clustering

## Greedy approach
Simply select the document set of every event, assign each document to a cluster and put the event to the cluster where majority of the documents belongs. This assumes that the documents are semantically similar enough to more or less agree on a cluster.

### Aperiodic

In [ ]:
aperiodic_event_clusters1 = [[] for _ in range(NUM_CLUSTERS)]

for event_id, event_docs in aperiodic_documents:
    if len(event_docs) == 0:
        print('Skipped event %d due to having empty document set' % event_id)
        continue
    
    docs_to_clusters = kmeans.predict(model[event_docs, :])

    mean = np.mean(docs_to_clusters)
    std = np.std(docs_to_clusters)
    mode = np.bincount(docs_to_clusters).argmax()
    median = np.median(docs_to_clusters)

    # Display document-cluster statistics.
    print('Event %2d: (mean: %.2f, std: %.2f, mode: %d, median: %d)' % (event_id, mean, std, mode, median))
    aperiodic_event_clusters1[mode].append(event_id)

In [ ]:
for i, event_cluster in enumerate(aperiodic_event_clusters1):
    print('Cluster %d' % i)

    for j, event in enumerate(event_cluster):
        words = [id2word[word_id] for word_id in aperiodic_events[event]]
        print('%02d. Event %2d (A): [%s]' % (j + 1, event, ', '.join(words)))

    print()

### Periodic

In [ ]:
periodic_event_clusters1 = [[] for _ in range(NUM_CLUSTERS)]

for event_id, event_docs in periodic_documents:
    if len(event_docs) == 0:
        print('Skipped event %d due to having empty document set' % event_id)
        continue

    docs_to_clusters = kmeans.predict(model[event_docs, :])
    
    mean = np.mean(docs_to_clusters)
    std = np.std(docs_to_clusters)
    mode = np.bincount(docs_to_clusters).argmax()
    median = np.median(docs_to_clusters)
    
    # Display document-cluster statistics.
    print('Event %2d: (mean: %.2f, std: %.2f, mode: %d, median: %d)' % (event_id, mean, std, mode, median))
    periodic_event_clusters1[mode].append(event_id)

In [ ]:
for i, event_cluster in enumerate(periodic_event_clusters1):
    print('Cluster %d' % i)

    for j, event in enumerate(event_cluster):
        words = [id2word[word_id] for word_id in periodic_events[event]]
        print('%02d. Event %2d (P): [%s]' % (j + 1, event, ', '.join(words)))

    print()

### Both

In [ ]:
with open('./greedy.txt', 'w') as f:
    for i, (aperiodic_cluster, periodic_cluster) in enumerate(zip(aperiodic_event_clusters1, periodic_event_clusters1)):
        print('Cluster %d' % i, file=f)

        j = 0

        for aperiodic_event in aperiodic_cluster:
            words = [id2word[word_id] for word_id in aperiodic_events[aperiodic_event]]
            print('%02d. Event %2d (A): [%s]' % (j + 1, aperiodic_event, ', '.join(words)), file=f)
            j += 1

        for periodic_event in periodic_cluster:
            words = [id2word[word_id] for word_id in periodic_events[periodic_event]]
            print('%02d. Event %2d (P): [%s]' % (j + 1, periodic_event, ', '.join(words)), file=f)
            j += 1

        print(file=f)

## Vector averaging
Compute an average vector from  all event documents and predict its cluster. Only minor differences when compared to the greedy approach, but it seems more robust.

### Aperiodic

In [ ]:
aperiodic_event_clusters2 = [[] for _ in range(NUM_CLUSTERS)]

for event_id, event_docs in aperiodic_documents:
    if len(event_docs) == 0:
        print('Skipped event %d due to having empty document set' % event_id)
        continue

    mean_doc = np.mean(model[event_docs, :], axis=0)
    mean_doc /= np.linalg.norm(mean_doc)
    cluster_id = kmeans.predict(mean_doc.reshape(1, -1))
    aperiodic_event_clusters2[cluster_id].append(event_id)

In [ ]:
for i, event_cluster in enumerate(aperiodic_event_clusters2):
    print('Cluster %d' % i)

    for j, event in enumerate(event_cluster):
        words = [id2word[word_id] for word_id in aperiodic_events[event]]
        print('%02d. Event %2d (A): [%s]' % (j + 1, event, ', '.join(words)))

    print()

### Periodic

In [ ]:
periodic_event_clusters2 = [[] for _ in range(NUM_CLUSTERS)]

for event_id, event_docs in periodic_documents:
    if len(event_docs) == 0:
        print('Skipped event %d due to having empty document set' % event_id)
        continue

    mean_doc = np.mean(model[event_docs, :], axis=0)
    mean_doc /= np.linalg.norm(mean_doc)
    cluster_id = kmeans.predict(mean_doc.reshape(1, -1))
    periodic_event_clusters2[cluster_id].append(event_id)

In [ ]:
for i, event_cluster in enumerate(periodic_event_clusters2):
    print('Cluster %d' % i)

    for j, event in enumerate(event_cluster):
        words = [id2word[word_id] for word_id in periodic_events[event]]
        print('%02d. Event %2d (P): [%s]' % (j + 1, event, ', '.join(words)))

    print()

### Both

In [ ]:
with open('./average.txt', 'w') as f:
    for i, (aperiodic_cluster, periodic_cluster) in enumerate(zip(aperiodic_event_clusters2, periodic_event_clusters2)):
        print('Cluster %d' % i, file=f)

        j = 0

        for aperiodic_event in aperiodic_cluster:
            words = [id2word[word_id] for word_id in aperiodic_events[aperiodic_event]]
            print('%02d. Event %2d (A): [%s]' % (j + 1, aperiodic_event, ', '.join(words)), file=f)
            j += 1

        for periodic_event in periodic_cluster:
            words = [id2word[word_id] for word_id in periodic_events[periodic_event]]
            print('%02d. Event %2d (P): [%s]' % (j + 1, periodic_event, ', '.join(words)), file=f)
            j += 1

        print(file=f)

# 10. Hierarchical clustering test

In [ ]:
model = lsi_gensim
aperiodic_means = []
event_titles = []

for event_id, event_docs in aperiodic_documents:
    if len(event_docs) == 0:
        print('Skipped event %d due to having empty document set' % event_id)
        print([id2word[word_id] for word_id in aperiodic_events[event_id]])
        continue

    mean_doc = np.mean(model[event_docs, :], axis=0)
    aperiodic_means.append(mean_doc)
    
    event = aperiodic_events[event_id]
    event_words = []
    
    for word_id in event:
        event_words.append(id2word[word_id])
        
    event_titles.append(', '.join(event_words))
    
aperiodic_means = np.array(aperiodic_means, dtype=float)
print(aperiodic_means.shape)

In [ ]:
from scipy.cluster.hierarchy import linkage
%time linkage_matrix = linkage(aperiodic_means, method='weighted', metric='cosine')
print(linkage_matrix.shape)

In [ ]:
from scipy.cluster.hierarchy import dendrogram

fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=event_titles);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() #show plot with tight layout
plt.savefig('cosine_weighted_clusters.png', dpi=200)